In [235]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [236]:
import numpy as np
from timer import Timer
import pickle

In [237]:
maxFloat16 = np.finfo(np.float16).max
neuronMax = 100
neuronMin = -100
neuronDtype = np.float32


class NeuralNetwork:
    @staticmethod
    def sigmoid(x, derivative=False):
        if derivative:
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

    @staticmethod
    def relu(x, derivative=False):
        if derivative:
            return 1.0 #* (x > 0)
        return x #np.maximum(0, x)


    # @staticmethod
    # def softmax(x):
    #     exps = np.exp(x - np.max(x))
    #     return exps / np.sum(exps, axis=1, keepdims=True)

    # @staticmethod
    # def softmaxDerivative(x):
    #     return x * (1 - x)

    def __init__(self, layerSizes, activationFunctions):
        self.layerSizes = layerSizes
        self.activationFunctions = activationFunctions
        self.layers = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i]).astype(neuronDtype)
            for i in range(len(layerSizes))
        ]
        self.weights = [
            np.random.uniform(
                low=neuronMin, high=neuronMax, size=layerSizes[i] * layerSizes[i + 1]
            ).astype(neuronDtype).reshape(layerSizes[i], layerSizes[i + 1])
            for i in range(len(layerSizes) - 1)
        ]
        self.biases = [
            np.random.uniform(low=neuronMin, high=neuronMax, size=layerSizes[i + 1]).astype(
                neuronDtype
            )
            for i in range(len(layerSizes) - 1)
        ]

    def loadInput(self, input):
        self.layers[0] = input.astype(neuronDtype)

    def readOutput(self):
        return self.layers[-1]

    def forward(self):
        for i in range(len(self.weights)):
            self.layers[i + 1] = self.activationFunctions[i](
                np.dot(self.layers[i], self.weights[i]) + self.biases[i]
            )

    def mseLoss(self, target, layer_no):
        return np.sum(np.square(target - self.layers[layer_no]))/2

    def backpropagation(self, target, learning_rate):
        output_error = target - self.layers[-1]

        for i in range(len(self.layers) - 2, -1, -1):
            # Calculate the derivative of the activation function
            derivative = self.activationFunctions[i](self.layers[i + 1], derivative=True)

            # Calculate the error of the current layer
            layer_error = np.dot(output_error * derivative, self.weights[i].T)

            # Calculate the adjustments for the weights and biases
            weight_adjustments = np.outer(self.layers[i], output_error)
            bias_adjustments = output_error

            # Update the weights and biases
            self.weights[i] += weight_adjustments * learning_rate
            self.biases[i] += bias_adjustments * learning_rate

            # Set the output error for the next iteration
            output_error = layer_error


In [238]:
x = NeuralNetwork([5, 5, 3], [NeuralNetwork.relu, NeuralNetwork.relu])
nnTimer = Timer(precision=3)

target = np.array([2, 3, 4])
input = np.random.rand(5)



# x = NeuralNetwork([30_000, 20, 20, 3], [NeuralNetwork.relu, NeuralNetwork.relu, NeuralNetwork.relu])
# nnTimer = Timer(precision=3)

# target = np.array([2, 3, 4])
# input = np.random.rand(30_000)



# for i in range(1):
#     nnTimer.tic()
#     x.loadInput(np.random.rand(30_000))
#     x.forward()
#     nnTimer.toc()
#     print(x.readOutput())
# nnTimer.stats()

In [239]:
x.loadInput(input)
x.forward()
print(x.readOutput())

[  8118.654  -40168.613   -1971.3448]


In [240]:
for i in range(100):
    nnTimer.tic()
    x.backpropagation(target, 0.01)
    x.loadInput(input)
    x.forward()
    print(x.readOutput())
    nnTimer.toc()

[ 4.7656166e+09 -2.3556076e+10 -1.1498066e+09]
[ 9.6511616e+26 -4.7704951e+27 -2.3285486e+26]
[ inf -inf -inf]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan nan nan]
[nan na

C:\Users\milos\AppData\Local\Temp\ipykernel_17412\2347882154.py:80: RuntimeWarning: overflow encountered in add
  self.weights[i] += weight_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_17412\2347882154.py:81: RuntimeWarning: overflow encountered in add
  self.biases[i] += bias_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_17412\2347882154.py:80: RuntimeWarning: invalid value encountered in add
  self.weights[i] += weight_adjustments * learning_rate
C:\Users\milos\AppData\Local\Temp\ipykernel_17412\2347882154.py:81: RuntimeWarning: invalid value encountered in add
  self.biases[i] += bias_adjustments * learning_rate


In [241]:
nnTimer.stats()

Total time: 0 s
Total calls: 100
Average time: 0 ms
